In [1]:
import pandas as pd
import numpy as np

In [16]:
#load data
data= pd.read_csv("1111_mn.csv")
data = data[['content', 'type_text']]

In [17]:

#shuppling data
data1=data.sample(frac=1)

In [18]:
data1

,content,type_text
2535,Сайн байна уу\r\nЗамын түгжрэлтэйгээр холбоото...,Гомдол
205,Voice/9235/Шинэчлэлийн Засгийн газрын үйл ажил...,Талархал
4607,Амгалангийн зам дагуу зам засаж байгаа. супер ...,Гомдол
4034,ХУДүүргийн хөдөлмөрийн хэлтэсийн дарга Алтанцэ...,Гомдол
869,"3-р эмнэлэгийн эмч Ганбаатар , Алтанхуяг, боол...",Талархал
...,...,...
1895,СХД-ийн Цамбагаравын Нэгдсэн эмнэлгийн Дотрын ...,Талархал
768,СХД 2-р амбулторийн шүдний тасгаар үйлчлүүлсэн...,Талархал
871,Өчигдөр БЗД-ийн 4-р хорооны 43-р байрны гадаа ...,Талархал
1794,СБД-н халамжийн мэргэжилтэн Энхтуул нь албан ү...,Талархал


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.


In [19]:
#setting all data to lower  
data1['content'] = data1['content'].apply(lambda x: x.lower())

#change label urlag and tech to integer like 0 and 1
data1['type_text'] = data1['type_text'].apply((lambda x: re.sub('Талархал','1',x)))
data1['type_text'] = data1['type_text'].apply((lambda x: re.sub('Гомдол','0',x)))
print(data1['content'])
print(data1[ data1['type_text'] == '1'].size)
print(data1[ data1['type_text'] == '0'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

    #vectorizing text data to int
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data1['content'].values)
X = tokenizer.texts_to_sequences(data1['content'].values)
X = pad_sequences(X, maxlen=128)

2535    сайн байна уу\r\nзамын түгжрэлтэйгээр холбоото...
205     voice/9235/шинэчлэлийн засгийн газрын үйл ажил...
4607    амгалангийн зам дагуу зам засаж байгаа. супер ...
4034    худүүргийн хөдөлмөрийн хэлтэсийн дарга алтанцэ...
869     3-р эмнэлэгийн эмч ганбаатар , алтанхуяг, боол...
                              ...                        
1895    схд-ийн цамбагаравын нэгдсэн эмнэлгийн дотрын ...
768     схд 2-р амбулторийн шүдний тасгаар үйлчлүүлсэн...
871     өчигдөр бзд-ийн 4-р хорооны 43-р байрны гадаа ...
1794    сбд-н халамжийн мэргэжилтэн энхтуул нь албан ү...
1088    нярайн түргэн тусламжийн 2-р ээлжийн их эмч да...
Name: content, Length: 5928, dtype: object
4866
6990


In [20]:
print(len(data1[ data1['type_text'] == '1']))
print(len(data1[ data1['type_text'] == '0']))

2433
3495


In [21]:
# building model for rnn sentimental analysis

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 128, 128)          256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 128, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
#declaring the train and test dataset.
Y = pd.get_dummies(data1['type_text']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)



(3971, 128) (3971, 2)
(1957, 128) (1957, 2)


In [23]:
#training
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

/home/nyma/virtual_env/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/7
 - 23s - loss: 0.2419 - accuracy: 0.9018
Epoch 2/7
 - 22s - loss: 0.0667 - accuracy: 0.9801
Epoch 3/7
 - 23s - loss: 0.0651 - accuracy: 0.9849
Epoch 4/7
 - 22s - loss: 0.0288 - accuracy: 0.9929
Epoch 5/7
 - 22s - loss: 0.0309 - accuracy: 0.9922
Epoch 6/7
 - 22s - loss: 0.0196 - accuracy: 0.9965
Epoch 7/7
 - 23s - loss: 0.0148 - accuracy: 0.9952


In [24]:
#Extracting a validation set, and measuring score and accuracy.
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.09
acc: 0.97


In [25]:

pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")



pos_acc 97.64150943396226 %
neg_acc 98.95833333333334 %


In [12]:

twt = [' Дүүргийн удирдлага, хотын удирдлага анхааралдаа авч засварлахгүй бол зүүн 3 аймаг, налайхын иргэд цаашлаад уг замаар зорчиж байгаа хүн бүрт маш хүндрэлтэй байна.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=128, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("Negative")
elif (np.argmax(sentiment) == 1):
    print("Positive")



Negative
